<h1>QA Dataset Generation</h1>

In [ ]:
!pip install transformers=="4.25.1" sentencepiece=="0.1.97" utoken=="0.1.8" nltk=="3.8.1" datasets=="2.8.0" torch=="1.13.1+cu116" numpy=="1.21.6" tqdm=="4.64.1" --quiet

In [ ]:
# import os
# os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
# os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [ ]:
import json
from pathlib import Path
import torch
from torch.utils.data import DataLoader
import time
import os
import random

## Model

In [ ]:
question_generator_args=dict(
    qg_model="./QGmodel/QGmodelfinal2",
    qa_model="./QA_models/banglabert-finetuned-squadbangla1-final",
    output_dir="./generated_questions",
    max_q_length=64,
    temperature=1.0,
    top_k=30,
    top_p=0.9,
    generated_answers_path="./QGdata",
    no_cuda=False
)


from types import SimpleNamespace

args = SimpleNamespace(**question_generator_args)

## Generate

In [ ]:
import logging as logger
import os
from tqdm.notebook import tqdm, trange
from functools import partial

import torch
from torch.utils.data import TensorDataset, SequentialSampler, DataLoader, RandomSampler
from torch.optim import AdamW
from transformers import (
    T5Config,
    T5Tokenizer,
    T5ForConditionalGeneration,
    BertForQuestionAnswering,
    AutoModelForQuestionAnswering,
    BertTokenizerFast,
    AutoTokenizer,
    BertConfig,
    get_linear_schedule_with_warmup,
)

from torch.utils.tensorboard import SummaryWriter

logger.basicConfig(
        format="%(asctime)s - %(levelname)s - %(name)s -   %(message)s",
        datefmt="%m/%d/%Y %H:%M:%S",
        level=logger.INFO
    )

if not os.path.exists(args.output_dir):
    os.makedirs(args.output_dir)

args.device = torch.device("cuda" if torch.cuda.is_available() and not args.no_cuda else "cpu")


qg_config = T5Config.from_pretrained(args.qg_model)

qg_tokenizer = T5Tokenizer.from_pretrained(args.qg_model)
# qg_tokenizer.add_tokens(['<sep>'])

qg_model = T5ForConditionalGeneration.from_pretrained(
    args.qg_model,
    config=qg_config
)

qg_model.resize_token_embeddings(len(qg_tokenizer))
qg_model.eval()
qg_model.to(args.device)


qa_tokenizer = AutoTokenizer.from_pretrained(args.qa_model)
qa_model = AutoModelForQuestionAnswering.from_pretrained(args.qa_model)
qa_model.eval()
qa_model.to(args.device)

In [ ]:
f = open("./BARDContextandAnswer.json")
text_data = json.load(f)
f.close()

In [ ]:
text_data['data'][4333]

{'title': 'economy',
 'context': 'এনসিসি ব্যাংকের উদ্যোক্তা পরিচালক মো. নুরুন নেওয়াজ সেলিম তৃতীয়বারের মতো ব্যাংকের চেয়ারম্যান নির্বাচিত হয়েছেন। ব্যাংকের আরেক উদ্যোক্তা পরিচালক এ এস এম মাঈন উদ্দিন মোনেম ভাইস চেয়ারম্যান নির্বাচিত হয়েছেন।নুরুন নেওয়াজ চট্টগ্রাম চেম্বারের জ্যেষ্ঠ সহসভাপতি ও চট্টগ্রামের ফেনী জেলা সমিতির সভাপতি। তিনি ইলেক্ট্রোমার্ট ও ট্রেড ইন্টারন্যাশনাল মার্কেটিংয়ের চেয়ারম্যান এবং সেন্ট্রাল ইনস্যুরেন্স কোম্পানির সাবেক চেয়ারম্যান।মাঈনুদ্দিন মোনেম বর্তমানে আবদুল মোনেম লিমিটেডের উপব্যবস্থাপনা পরিচালক, নোভাস ফার্মাসিউটিক্যালস ও এএম সিকিউরিটিজ অ্যান্ড ফাইন্যান্সিয়াল সার্ভিসেসের প্রধান নির্বাহী। ।',
 'sent_list': ['এনসিসি ব্যাংকের উদ্যোক্তা পরিচালক মো. নুরুন নেওয়াজ সেলিম তৃতীয়বারের মতো ব্যাংকের চেয়ারম্যান নির্বাচিত হয়েছেন।',
  'ব্যাংকের আরেক উদ্যোক্তা পরিচালক এ এস এম মাঈন উদ্দিন মোনেম ভাইস চেয়ারম্যান নির্বাচিত হয়েছেন।',
  'নুরুন নেওয়াজ চট্টগ্রাম চেম্বারের জ্যেষ্ঠ সহসভাপতি ও চট্টগ্রামের ফেনী জেলা সমিতির সভাপতি।',
  'তিনি ইলেক্ট্রোমার্ট ও ট্রেড ইন্টারন্যাশনাল মার্ক

In [ ]:
# ids = []
# qg_data = []
# ans_data = []
# for di in range(len(text_data['data'])):
#     data = text_data['data'][di]
#     for answer in data['answers']:
#         if len(data['sent_list'][answer['sent_with_ans']])>1 and len(answer['text'])>1:
#             text = data['sent_list'][answer['sent_with_ans']]+' <sep> '+answer['text']
#             qg_data.append(text)
#             ans_data.append(answer)
#             ids.append(di)

In [ ]:
ids = []
qg_data = []
ans_data = []
for di in range(len(text_data['data'])):
    data = text_data['data'][di]
    for sent in data['sent_list']:
        if(len(sent)>1):
            text = sent+' <sep> '+"impossible"
            qg_data.append(text)
            ans_data.append("impossible")
            ids.append(di)

In [ ]:
import math
m = len(qg_data)
mini_batch_size = 32
mini_batches = []
num_complete_minibatches = math.floor(m / mini_batch_size) 
for k in range(0, num_complete_minibatches):

    mini_batch_X = qg_data[k*mini_batch_size:(k+1)*mini_batch_size]
    mini_batches.append(mini_batch_X)

if m % mini_batch_size != 0:
    mini_batch_X = qg_data[num_complete_minibatches*mini_batch_size:]
    mini_batches.append(mini_batch_X)

In [ ]:
questions = []
print('No. of batches: ', len(mini_batches))

for batch in tqdm(mini_batches):
  encodings = qg_tokenizer(batch, return_tensors="pt",max_length=512,truncation=True, padding=True)
  all_input_ids = torch.tensor(encodings['input_ids'], dtype=torch.long)
  outputs = qg_model.generate(all_input_ids.to(args.device),
                    max_length=50, 
                    num_beams=5, 
                    no_repeat_ngram_size=2, 
                    num_return_sequences=1, 
                    early_stopping=True)
  batch_ques = outputs.detach().cpu().numpy()
  batch_ques = batch_ques.reshape(len(batch),1,-1)
  
  for e in batch_ques:
    context_ques = []
    for aques in e:
      context_ques.append(qg_tokenizer.decode(aques, skip_special_tokens=True))
    questions.append(context_ques)
  

No. of batches:  6605


  0%|          | 0/6605 [00:00<?, ?it/s]

/tmp/ipykernel_10778/3621848040.py:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  all_input_ids = torch.tensor(encodings['input_ids'], dtype=torch.long)


In [ ]:
examples = []
for i in range(len(ids)):
    examples.append({'id': ids[i],
                        'context':text_data['data'][ids[i]]['context'],
                        'sent_with_ans': qg_data[i].split(" <sep> ")[0],
                        'questions':questions[i],
                        'answer': ans_data[i]
    })

In [ ]:
examples[102]

{'id': 10,
 'context': 'সম্প্রতি প্রকাশিত ‘১৯৭১: ভেতরে বাইরে’ বইয়ে মুজিব বাহিনী নিয়ে কটূক্তি করার অভিযোগে লেখক এ কে খন্দকারের বিরুদ্ধে ব্রাহ্মণবাড়িয়ায় মানহানির মামলা হয়েছে। আজ বুধবার বেলা একটার দিকে ইছহাক ভূঁইয়া নামের এক ব্যক্তি ব্রাহ্মণবাড়িয়ার জ্যেষ্ঠ বিচারিক হাকিম আদালতে মামলাটি করেন। বাদী ইছহাক ভূঁইয়া আদালতে নিজেকে ব্রাহ্মণবাড়িয়া অঞ্চলের মুজিব বাহিনীর কমান্ডার দাবি করেছেন। মামলাটি গ্রহণ করে ১৫ সেপ্টেম্বরের মধ্যে ইছহাককে মুক্তিযোদ্ধার সনদ আদালতে জমা দেওয়ার নির্দেশ দেন আদালত। সনদ দেখে আদালত পরবর্তী আদেশ দেবেন। বাদীপক্ষের আইনজীবী তানভীর ভূঁইয়া বলেন, দণ্ডবিধি আইনের ৫০০ ও ৫০১ ধারায় মামলা করা হয়। যোগাযোগ করা হলে ইছহাক ভূঁইয়া প্রথম আলোকে বলেন, মুক্তিযুদ্ধ চলাকালে বাংলাদেশ বাহিনীর ডেপুটি চিফ অব স্টাফ ও সাবেক পরিকল্পনামন্ত্রী এ কে খন্দকার তাঁর বইয়ে মুজিব বাহিনী নিয়ে কটূক্তি করায় এ মামলা করা হয়। তিনি দাবি করেন, এ কে খন্দকার উদ্দেশ্যপ্রণোদিতভাবে ভুল পথে চলছেন ও বর্তমান সরকারকে কলঙ্কিত করতে এ ধরনের মিথ্যা অপপ্রচার চালাচ্ছেন।',
 'sent_with_ans': 'মামলাটি গ্রহণ করে ১৫ সেপ্টেম

In [ ]:
# temp_data = {}
# temp_data['data'] =examples
# final_out_file = open('./BARDQAimpossibletempDataset.json', "w")
# json.dump(temp_data, final_out_file, indent = 4) # save whole data replace parts later
# final_out_file.close()

In [ ]:
f = open("./BARDQAtempDataset.json")
temp_data = json.load(f)
f.close()

examples = temp_data['data']

In [ ]:
examples[59]

{'id': 7,
 'context': 'এবার ঝিনাইদহ সদর উপজেলা মুক্তিযোদ্ধা কমান্ড কার্যালয়ে এক মুক্তিযোদ্ধাকে পিটিয়ে আহত করা হয়েছে। গতকাল রোববার বিকেলে এ ঘটনা ঘটে। আহত মুক্তিযোদ্ধার নাম ইউনুছ আলী। বাড়ি ভুটিয়ারগাতি গ্রামে। তাঁকে সদর হাসপাতালে ভর্তি করা হয়েছে। সদর উপজেলার মুক্তিযোদ্ধা কমান্ডার সিদ্দিক আহাম্মেদ প্রথম আলোকে বলেন, ‘বিকেলে কার্যালয়ে এসে ভুটিয়ারগাতি গ্রামের তিন তরুণ মুক্তিযোদ্ধাদের নামের তালিকা দেখতে চায়। তারা এক ব্যক্তির নাম উল্লেখ করে তিনি কীভাবে মুক্তিযোদ্ধার তালিকায় স্থান পেয়েছেন, তা জানতে চায়। আমি তাদের তালিকা দিতে অপারগতা প্রকাশ করি। এতে তারা ক্ষুব্ধ হয়। পরে মুক্তিযোদ্ধা ইউনুছ আলীকে বাঁশ দিয়ে পিটিয়ে আহত করে।’ ইউনুছ আলী বলেন, ‘আমি মুক্তিযোদ্ধা কার্যালয়ের ভেতরে ছিলাম। পরে ওই তরুণেরা তালিকা নিয়ে বাগ্\u200cবিতণ্ডা করলে কক্ষ থেকে বেরিয়ে আসি। হঠাৎ করে তরুণেরা অফিসের ভেতর থেকে বের হয়ে আমার ওপর হামলা চালায়। চিৎকার করলে আশপাশের লোকজন ছুটে আসে। তখন হামলাকারীরা পালিয়ে যায়।’ সদর থানার ভারপ্রাপ্ত কর্মকর্তা (ওসি) হরেন্দ্রনাথ সরকার বলেন, এ ঘটনায় জড়িত সন্দেহে হিল্লোল নামের এক তরুণকে আটক করা হয়ে

In [ ]:
import string
from collections import Counter
import numpy as np
punc = '''!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~—।–'''

def f1_score(prediction, ground_truth):
    prediction_tokens = prediction.translate(str.maketrans('', '', punc)).split()
    ground_truth_tokens = ground_truth.translate(str.maketrans('', '', punc)).split()
    common = Counter(prediction_tokens) & Counter(ground_truth_tokens)
    num_same = sum(common.values())
    if num_same == 0:
        return 0
    precision = 1.0 * num_same / len(prediction_tokens)
    recall = 1.0 * num_same / len(ground_truth_tokens)
    f1 = (2 * precision * recall) / (precision + recall)
    return f1

def valid_questions(args, qa_tokenizer, qa_model, examples):
    n_best_size = 3
    max_answer_length = 30
    i=0
    dataset = []
    for example in tqdm(examples):
        context = example['sent_with_ans']
        i+=1
        qas = None
        imp_qas = None
        for ques in example['questions']:
            encodings = qa_tokenizer(context, ques, max_length=512, truncation=True, padding=True, return_offsets_mapping=True)
            input_ids = torch.tensor([encodings.input_ids]).to(args.device)
            attention_mask = torch.tensor([encodings.attention_mask]).to(args.device)
            model_output = qa_model(input_ids, attention_mask=attention_mask)
            start_token_scores = model_output[0].detach().cpu().numpy().flatten()
            end_token_scores = model_output[1].detach().cpu().numpy().flatten()
            valid_answers = []
            start_indexes = np.argsort(start_token_scores)[-1 : -n_best_size - 1 : -1].tolist()
            end_indexes = np.argsort(end_token_scores)[-1 : -n_best_size - 1 : -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    if (
                        start_index >= encodings.token_type_ids.index(1)
                        or end_index >= encodings.token_type_ids.index(1)
                        or encodings.offset_mapping[start_index] is None
                        or encodings.offset_mapping[end_index] is None
                    ):
                        continue
                    # Don't consider answers with a length that is either < 0 or > max_answer_length.
                    if end_index < start_index or end_index - start_index + 1 > max_answer_length:
                        continue

                    start_char = encodings.offset_mapping[start_index][0]
                    end_char =encodings.offset_mapping[end_index][1]

                    valid_answers.append(
                        {
                            "score": start_token_scores[start_index] + start_token_scores[end_index],
                            "text": context[start_char: end_char]
                        }
                    )
            # print(valid_answers)      
            if len(valid_answers) > 0:
                best_answer = sorted(valid_answers, key=lambda x: x["score"], reverse=True)[0]
            else:
                # In the very rare edge case we have not a single non-null prediction, we create a fake prediction to avoid
                # failure.
                best_answer = {"text": "", "score": 0.0}
            # print(ques, best_answer, example['answer'], f1_score(best_answer['text'], example['answer']))
            #similarity = f1_score(best_answer['text'], example['answer']['text'])
#             if (imp_qas is None and similarity<=0.3 and best_answer['score']<=3.0) or (imp_qas is not None and best_answer['score']<imp_qas['score']):
#                 imp_qas = {'question': ques, 'answer': "", 'score': best_answer['score']} 
            if (qas is None and best_answer['score']<3.0) or (qas is not None and best_answer['score']<qas['score']):
                qas = {'question': ques, 'answer': example['answer'], 'score': best_answer['score']} 
        dataset.append({'id': example['id'],'qas': qas})
    
    return dataset


In [ ]:
new_dataset = valid_questions(args, qa_tokenizer, qa_model, examples)

  0%|          | 0/211332 [00:00<?, ?it/s]

In [ ]:
new_dataset[20]

{'id': 1,
 'qas': {'question': 'কানাডিয়ান তদন্ত সংস্থা কতজনকে জিজ্ঞাসাবাদ করেছে?',
  'answer': 'impossible',
  'score': -0.17466894}}

In [ ]:
processed_dataset = {}
for new_data in new_dataset:
    if new_data['id'] not in processed_dataset:
        processed_dataset[new_data['id']] = {}
        processed_dataset[new_data['id']]['title'] = text_data['data'][new_data['id']]['title']
        processed_dataset[new_data['id']]['context'] = text_data['data'][new_data['id']]['context']
        processed_dataset[new_data['id']]['qas'] = []
        

    processed_dataset[new_data['id']]['qas'].append(new_data['qas'])

In [ ]:
final_dataset = []
for p_data in processed_dataset:
    final_dataset.append({'id':p_data,'title': processed_dataset[p_data]['title'], 'context': processed_dataset[p_data]['context'], 'qas': processed_dataset[p_data]['qas']})

In [ ]:
for datas in final_dataset:
  datas['qas'] = [qas for qas in datas['qas'] if qas is not None]
  for qas in datas['qas']:
      qas['score'] = float(qas['score'])
      

In [ ]:
data = {}
data['data'] = final_dataset

In [ ]:
for valid_qa in temp_data['data']:
    data['data'][valid_qa['id']]['qas'].append({'question':valid_qa['questions'][0],'answer':valid_qa['answer']})

In [ ]:
for all_qa in data['data']:
    all_qa['sent_list'] = text_data['data'][all_qa['id']]['sent_list']

In [ ]:
from random import sample,shuffle
for all_qa in data['data']:
    imp_qa = [imp for imp in all_qa['qas'] if imp['answer']=='impossible']
    pos_qa = [imp for imp in all_qa['qas'] if imp['answer']!='impossible']
    imp_qa = sample(imp_qa,min(len(imp_qa),int(0.3*len(pos_qa))))
    for qa in imp_qa:
        qa['answer'] = {'text': [],'answer_start':[]}
    for qa in pos_qa:
        answer_start = len(" ".join(all_qa['sent_list'][0:qa['answer']['sent_with_ans']]))+1+all_qa['sent_list'][qa['answer']['sent_with_ans']].index(qa['answer']['text'])
        qa['answer'] = {'text':[qa['answer']['text']], 'answer_start':[answer_start]}
    qas = []
    qas.extend(pos_qa)
    qas.extend(imp_qa)
    shuffle(qas)
    all_qa['qas']=qas
    all_qa['context'] = " ".join(all_qa['sent_list'])

In [ ]:
data['data'] = [qa_data for qa_data in data['data'] if len(qa_data['qas'])>3]

In [ ]:
final_out_file = open('./BARDFINALQADataset.json', "w")
json.dump(data, final_out_file, indent = 4) # save whole data replace parts later
final_out_file.close()

In [ ]:
f = open("./BARDFINALQADataset.json")
data = json.load(f)
f.close()


In [ ]:
data['data'][0]

{'id': 0,
 'title': 'state',
 'context': 'দৈনিক ইনকিলাব-এর কূটনৈতিক প্রতিবেদক আহমেদ আতিকের দুই দিনের রিমান্ড মঞ্জুর করেছেন আদালত। একই সঙ্গে বার্তা সম্পাদক রবিউল্লাহ রবি ও উপপ্রধান প্রতিবেদক রফিক মোহাম্মদের জামিন আবেদন নাকচ করে তাঁদের কারাগারে পাঠানোর নির্দেশ দিয়েছেন। গতকাল সোমবার ঢাকা মহানগর হাকিম এস এম আশিকুর রহমান এ আদেশ দেন। গত শুক্রবার তিনজনকে আদালতে হাজির করে কেবল আহমেদ আতিককে জিজ্ঞাসাবাদের জন্য পাঁচ দিনের রিমান্ডের আবেদন জানায় পুলিশ। আতিক, বার্তা সম্পাদক ও উপপ্রধান প্রতিবেদকের জামিন চান তাঁদের আইনজীবীরা। ঢাকা মহানগর হাকিম মোহাম্মদ ইউনুস খান ওই দিন রিমান্ড ও জামিনের শুনানি গ্রহণ না করে গতকাল তা শুনানির দিন ধার্য করেছিলেন। গতকাল রাষ্ট্রপক্ষের আইনজীবী নজরুল ইসলাম রিমান্ড শুনানিতে আদালতকে বলেন, ১৬ জানুয়ারি ‘সাতক্ষীরায় যৌথ বাহিনীর অপারেশনে ভারতীয় সেনাবাহিনীর সহায়তা’ শিরোনামে প্রকাশিত সংবাদের মাধ্যমে পত্রিকাটি সরকার উৎখাতের ষড়যন্ত্র হিসেবে কাজ করেছে। আসামি আহমেদ আতিক মিথ্যা ও অসত্য তথ্যের ভিত্তিতে প্রতিবেদনটি প্রস্তুত করেন। এই ষড়যন্ত্রের পেছনে কারা কারা আছে, সে ব্যাপারে জানতে আসামিকে র

In [ ]:
# temp_qa = data['data'][0]
# qa = temp_qa['qas'][-1]
# ans_ind = temp_qa['sent_list'][qa['answer']['sent_with_ans']].index(qa['answer']['text'])
# ans_ind = len(" ".join(temp_qa['sent_list'][0:qa['answer']['sent_with_ans']]))+ans_ind+1
# " ".join(temp_qa['sent_list'])[ans_ind]

In [ ]:
categories = ['state','economy','international','sports','entertainment']
for category in categories:
    count = 0
    for qa_data in data['data']:
        if qa_data['title'] == category:
            count+=len(qa_data['qas'])
    print(category,"---->",count)

state ----> 29901
economy ----> 24964
international ----> 38782
sports ----> 40731
entertainment ----> 43634


In [ ]:
for art in data['data']:
    print(art['id'])
    for qa in art['qas']:
        if qa['answer']['text']!=[]:
            print(qa['question'])
            print(qa['answer'])
            print(art['context'][qa['answer']['answer_start'][0]:qa['answer']['answer_start'][0]+len(qa['answer']['text'][0])])
    break

0
আহমেদ আতিককে জিজ্ঞাসাবাদের জন্য কত সময় দেওয়া হয়েছিল?
{'text': ['পাঁচ দিনের'], 'answer_start': [362]}
পাঁচ দিনের
কে এই আদেশ দিয়েছে?
{'text': ['এস এম আশিকুর রহমান'], 'answer_start': [257]}
এস এম আশিকুর রহমান
কোন আবেদন আদালত প্রত্যাখ্যান করেছে?
{'text': ['জামিনের আবেদন'], 'answer_start': [1161]}
জামিনের আবেদন
কেন এটা প্রকাশ করা হয়নি?
{'text': ['সরকারের ভাবমূর্তি নষ্ট করতে'], 'answer_start': [1094]}
সরকারের ভাবমূর্তি নষ্ট করতে
কোন দুই সাংবাদিককে তাদের আইনজীবীরা জামিনের জন্য আবেদন করেছিলেন?
{'text': ['বার্তা সম্পাদক ও উপপ্রধান প্রতিবেদকের'], 'answer_start': [408]}
বার্তা সম্পাদক ও উপপ্রধান প্রতিবেদকের
কতজন সাংবাদিককে গ্রেফতার করা হয়েছে?
{'text': ['বার্তা সম্পাদকসহ তিন সাংবাদিককে'], 'answer_start': [1370]}
বার্তা সম্পাদকসহ তিন সাংবাদিককে
কোন আইনে পুলিশ ওয়ারী থানায় মামলা করেছে?
{'text': ['তথ্য ও যোগাযোগপ্রযুক্তি আইনে'], 'answer_start': [1422]}
তথ্য ও যোগাযোগপ্রযুক্তি আইনে
কোন বার্তা সম্পাদককে কারাগারে পাঠানো হয়েছে?
{'text': ['রবিউল্লাহ রবি'], 'answer_start': [113]}
রবিউল্লাহ রবি
কো

In [ ]:
temp_train = {}
temp_train['data']= []
temp_val = {}
temp_val['data']= []
temp_test = {}
temp_test['data']= []

In [ ]:
dom_data = [qa_data for qa_data in data['data'] if qa_data['title'] == 'entertainment'] #Do this for every categories ['state','economy','international','sports','entertainment']
train_dom = dom_data[:int(0.8*len(dom_data))]
val_dom = dom_data[int(0.8*len(dom_data)):int(0.9*len(dom_data))]
test_dom = dom_data[int(0.9*len(dom_data)):]
temp_train['data'].extend(train_dom)
temp_val['data'].extend(val_dom)
temp_test['data'].extend(test_dom)
print(len(temp_train['data']))
print(len(temp_val['data']))
print(len(temp_test['data']))

12797
1600
1600


In [ ]:
# 12797
# 1600
# 1600

In [ ]:
train = {}
train['data']= []
val = {}
val['data']= []
test = {}
test['data']= []

In [ ]:
for datum in temp_train['data']:
    i=0
    for qa in datum['qas']:
        i+=1;
        qa_pair = {}
        qa_pair['id'] = str(datum['id'])+"_"+str(i)
        qa_pair['title'] = datum['title']
        qa_pair['context'] = datum['context']
        qa_pair['question'] = qa['question']
        qa_pair['answer'] = qa['answer']
        train['data'].append(qa_pair)

out_file = open('./train_BARDQA.json', "w")
json.dump(train, out_file, indent = 4) # save whole data replace parts later
out_file.close()

In [ ]:
for datum in temp_val['data']:
    i=0
    for qa in datum['qas']:
        i+=1;
        qa_pair = {}
        qa_pair['id'] = str(datum['id'])+"_"+str(i)
        qa_pair['title'] = datum['title']
        qa_pair['context'] = datum['context']
        qa_pair['question'] = qa['question']
        qa_pair['answer'] = qa['answer']
        val['data'].append(qa_pair)

out_file = open('./val_BARDQA.json', "w")
json.dump(val, out_file, indent = 4) # save whole data replace parts later
out_file.close()

In [ ]:
for datum in temp_test['data']:
    i=0
    for qa in datum['qas']:
        i+=1;
        qa_pair = {}
        qa_pair['id'] = str(datum['id'])+"_"+str(i)
        qa_pair['title'] = datum['title']
        qa_pair['context'] = datum['context']
        qa_pair['question'] = qa['question']
        qa_pair['answer'] = qa['answer']
        test['data'].append(qa_pair)

out_file = open('./test_BARDQA.json', "w")
json.dump(test, out_file, indent = 4) # save whole data replace parts later
out_file.close()

In [ ]:
test['data'][20] #['context'][test['data'][20]['answer']['answer_start'][0]]

{'id': '3601_5',
 'title': 'state',
 'context': 'কুষ্টিয়ার দৌলতপুর উপজেলার আড়িয়া গ্রামের মুঠোফোন মেরামতকারী আজম আলীকে (২২) খুন করেছে দুর্বৃত্তরা। গতকাল শুক্রবার বিকেল চারটার দিকে আড়িয়া গ্রামের পাশের দিঘলকান্দি গ্রামের মাঠ থেকে তাঁর লাশ উদ্ধার করে পুলিশ। দৌলতপুর থানার ভারপ্রাপ্ত কর্মকর্তা (ওসি) এনামুল হক বলেন, হত্যাকাণ্ডের সঙ্গে জড়িত সন্দেহে আড়িয়া গ্রামের স্বপন আলীকে (৩৫) গ্রেপ্তার করা হয়েছে। প্রাথমিক জিজ্ঞাসাবাদে তিনি হত্যাকাণ্ডে জড়িত থাকার কথা স্বীকার করেছেন। স্বপন বলেছেন, তিনি আজমের কাছ থেকে এক লাখ ৩০ হাজার টাকা ঋণ নেন। সুদের বিনিময়ে এ রকম আরও তিন-চারজন আজমের কাছ থেকে ঋণ নেন। আজম ওই টাকা পরিশোধের জন্য চাপ দিলে তাঁরা বৃহস্পতিবার রাতে আজমকে বাড়ি থেকে ডেকে নিয়ে হত্যা করেন। এরপর পাশের দিঘলকান্দি গ্রামের মাঠে পুঁতে রাখেন। এলাকাবাসী সূত্র জানায়, বৃহস্পতিবার রাতে অনেক খোঁজ করেও আজমকে খুঁজে পায়নি তাঁর পরিবার। গতকাল দুপুরে দিঘলকান্দি গ্রামের এক কৃষক মাঠে খেতের ভেতর মাটিচাপা দেওয়া উঁচু কিছু দেখে পুলিশকে খবর দেন। খবর পেয়ে দৌলতপুর থানার পুলিশ সেখান থেকে একটি মৃতদেহ উদ্ধার করে। পরে আজমের

In [ ]:
# 142536
# 17861
# 17615

In [ ]:
import string
from collections import Counter
import numpy as np
punc = '''!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~—।–'''
i=0
n_best_size = 3
max_answer_length = 30
for example in tqdm(examples):
    context = example['sent_with_ans']
    i+=1
    for ques in example['questions']:
        encodings = qa_tokenizer(context, ques, max_length=512, truncation=True, padding=True, return_offsets_mapping=True)
        input_ids = torch.tensor([encodings.input_ids]).to(args.device)
        attention_mask = torch.tensor([encodings.attention_mask]).to(args.device)
        model_output = qa_model(input_ids, attention_mask=attention_mask)
        start_token_scores = model_output[0].detach().cpu().numpy().flatten()
        end_token_scores = model_output[1].detach().cpu().numpy().flatten()
        valid_answers = []
        start_indexes = np.argsort(start_token_scores)[-1 : -n_best_size - 1 : -1].tolist()
        end_indexes = np.argsort(end_token_scores)[-1 : -n_best_size - 1 : -1].tolist()
        for start_index in start_indexes:
            for end_index in end_indexes:
                if (
                    start_index >= encodings.token_type_ids.index(1)
                    or end_index >= encodings.token_type_ids.index(1)
                    or encodings.offset_mapping[start_index] is None
                    or encodings.offset_mapping[end_index] is None
                ):
                    continue
                # Don't consider answers with a length that is either < 0 or > max_answer_length.
                if end_index < start_index or end_index - start_index + 1 > max_answer_length:
                    continue

                start_char = encodings.offset_mapping[start_index][0]
                end_char =encodings.offset_mapping[end_index][1]

                valid_answers.append(
                    {
                        "score": start_token_scores[start_index] + start_token_scores[end_index],
                        "text": context[start_char: end_char]
                    }
                )
        if len(valid_answers) > 0:
            best_answer = sorted(valid_answers, key=lambda x: x["score"], reverse=True)[0]
        else:
            # In the very rare edge case we have not a single non-null prediction, we create a fake prediction to avoid
            # failure.
            best_answer = {"text": "", "score": 0.0}
        print("-----------------------------------")
        print(best_answer)
        print(example['answer']['text'])
        print(f1_score(best_answer['text'], example['answer']['text']))
        print(example['sent_with_ans'])
        print(ques)
    if i>20:
        break

  0%|          | 0/150439 [00:00<?, ?it/s]

-----------------------------------
{'score': 4.716404, 'text': 'আহমেদ'}
আহমেদ আতিকের
0.6666666666666666
দৈনিক ইনকিলাব-এর কূটনৈতিক প্রতিবেদক আহমেদ আতিকের দুই দিনের রিমান্ড মঞ্জুর করেছেন আদালত।
ইনকিলাব পত্রিকার কূটনৈতিক সংবাদদাতা কে ছিলেন?
-----------------------------------
{'score': 5.576952, 'text': 'আহমেদ'}
আহমেদ আতিকের
0.6666666666666666
দৈনিক ইনকিলাব-এর কূটনৈতিক প্রতিবেদক আহমেদ আতিকের দুই দিনের রিমান্ড মঞ্জুর করেছেন আদালত।
ইনকিলাব পত্রিকার কূটনৈতিক রিপোর্টার কে ছিলেন?
-----------------------------------
{'score': 4.0774927, 'text': 'আহমেদ'}
আহমেদ আতিকের
0.6666666666666666
দৈনিক ইনকিলাব-এর কূটনৈতিক প্রতিবেদক আহমেদ আতিকের দুই দিনের রিমান্ড মঞ্জুর করেছেন আদালত।
ইনকিলাব-এর কূটনৈতিক সংবাদদাতা কে ছিলেন?
-----------------------------------
{'score': 5.3662534, 'text': 'আহমেদ'}
আহমেদ আতিকের
0.6666666666666666
দৈনিক ইনকিলাব-এর কূটনৈতিক প্রতিবেদক আহমেদ আতিকের দুই দিনের রিমান্ড মঞ্জুর করেছেন আদালত।
ইনকিলাব-এর কূটনৈতিক রিপোর্টার কে ছিলেন?
-----------------------------------
{'score': 6.153358

-----------------------------------
{'score': -0.58293676, 'text': 'নাইকো রিসোর্সের'}
নাইকো রিসোর্সের
1.0
কানাডার কোম্পানি নাইকো রিসোর্সের কাছ থেকে ঘুষ গ্রহণের মামলায় সাবেক জ্বালানি উপদেষ্টা ও আমার দেশ পত্রিকার ভারপ্রাপ্ত সম্পাদক মাহমুদুর রহমানকে জিজ্ঞাসাবাদ করেছে কানাডার তদন্ত সংস্থা।
কোন কানাডিয়ান কোম্পানির কাছ থেকে ঘুষ গ্রহণের অভিযোগে মাহমুদুর রহমানকে জিজ্ঞাসাবাদ করা হয়েছিল?
-----------------------------------
{'score': 6.8304825, 'text': 'মহানগর হাকিম রেজাউল'}
মহানগর হাকিম রেজাউল করিমের
0.8571428571428571
গতকাল রোববার ঢাকার মুখ্য মহানগর হাকিম আদালতে মহানগর হাকিম রেজাউল করিমের উপস্থিতিতে তাঁকে জিজ্ঞাসাবাদ করা হয়।
কার উপস্থিতিতে তাকে জিজ্ঞাসাবাদ করা হয়েছিল?
-----------------------------------
{'score': 3.6544485, 'text': 'মহানগর হাকিম'}
মহানগর হাকিম রেজাউল করিমের
0.6666666666666666
গতকাল রোববার ঢাকার মুখ্য মহানগর হাকিম আদালতে মহানগর হাকিম রেজাউল করিমের উপস্থিতিতে তাঁকে জিজ্ঞাসাবাদ করা হয়।
কে তাকে জিজ্ঞাসাবাদ করেছে?
-----------------------------------
{'score': 6.9003987, 'text':

In [ ]:
from pprint import pprint
pprint(final_dataset['data'][0])

Streaming output truncated to the last 5000 lines.
             'ঠাকুরকে ছয় মেরে গেছেন ৪৯ রানে, পরের ওভারে কুলদীপ যাদবকে চার মেরে '
             'অর্ধশতক পেয়েছেন ৩৬ বলে। পরের পঞ্চাশে বাটলার সময় নিয়েছেন মাত্র ২১ '
             'বল। সব মিলিয়ে একশ পর্যন্ত ইনিংসে ৮টি করে চার ও ছয়। বাটলার অবশেষে '
             'থেমেছেন মোস্তাফিজুর রহমানের করা ১৯তম ওভারের শেষ বলে। ডেভিড '
             'ওয়ার্নারের হাতে ধরা পড়ার আগে ৬৫ বলে করেছেন ১১৬ রান, মেরেছেন ৯টি '
             'করে চার ও ছয়। ৭ ম্যাচে শেষেই বাটলারের রান এখন ৪৯১, দুইয়ে থাকা '
             'লোকেশ রাহুলের রান ২৬৫। বাটলারের শতকের সঙ্গে দেবদূত পাডিক্কালের '
             '৩৫ বলে ৫৪, অধিনায়ক সঞ্জু স্যামসনের ১৯ বলে ৪৬ রানে রাজস্থান '
             'তুলেছে ২ উইকেটে ২২২ রান, এ মৌসুমে কোনো দলের যা সর্বোচ্চ। শেষ ৩ '
             'ওভারেই রাজস্থান তুলেছে ৫৫ রান। আইপিএলের এ মৌসুমে দ্বিতীয় ম্যাচেই '
             'মুম্বাই ইন্ডিয়ানসের বিপক্ষে শতক পান বাটলার। ৩ ম্যাচে পর কলকাতার '
             'বিপক্ষে করলেন আরেকটি। এ শতকটি তাই বাটলারের টানা দ্বিতীয়। আইপিএলে 